# Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd
import functools
from nltk.tokenize import sent_tokenize
from pywsd.utils import lemmatize_sentence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score



Warming up PyWSD (takes ~10 secs)... took 4.044273138046265 secs.


# Taking a chunk of file and reading the chunk

In [2]:
chunk_iter = pd.read_csv("D:/QTInternship/aws_review_sofware_dataset/aws_review_sofware_dataset.csv",chunksize=20000)

#chunk_iter = pd.read_csv('your_file.csv', chunksize=1000)
df = next(chunk_iter)  # Read the first chunk

In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

In [5]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

In [6]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [7]:
df["words_sentences"] = "default"

In [8]:
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b: str(a)+str(" ")+str(b), df.loc[k,"words"])

In [9]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

# Splitting the data and Encoding the target variable


In [10]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [11]:
df_y=df["verified"]

In [12]:
le=LabelEncoder()

In [13]:
df_y_1=pd.DataFrame(df_y)

In [14]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [15]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


# Splitting the data into Train and Test data

In [16]:
X_Train,X_Test,Y_Train,Y_Test = train_test_split(dense,df_y_enc,test_size=0.3,random_state=45,stratify=df_y_enc)


# Model Building

In [17]:
RFC = RandomForestClassifier(random_state=45,n_estimators=1000)
RFC.fit(X_Train, Y_Train)

Y_pred_Train=RFC.predict(X_Train)
accuracy_train = accuracy_score(Y_Train, Y_pred_Train)
print(f"Accuracy of Training data is: {accuracy_train:.2f}")

print(confusion_matrix(Y_Train,Y_pred_Train))

# Generate a classification report
report = classification_report(Y_Train, Y_pred_Train)
print(f"Classification Report:\n{report}")



Y_pred_Test=RFC.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\AppData\Local\Temp\ipykernel_28932\2880767766.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RFC.fit(X_Train, Y_Train)


Accuracy of Training data is: 1.00
[[9571    7]
 [   0 4422]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9578
           1       1.00      1.00      1.00      4422

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000

Accuracy of Test data is: 0.79
[[3832  273]
 [ 958  937]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.93      0.86      4105
           1       0.77      0.49      0.60      1895

    accuracy                           0.79      6000
   macro avg       0.79      0.71      0.73      6000
weighted avg       0.79      0.79      0.78      6000



In [18]:
MNB = MultinomialNB()
MNB.fit(X_Train, Y_Train)

Y_pred_Train=MNB.predict(X_Train)
accuracy_train = accuracy_score(Y_Train, Y_pred_Train)
print(f"Accuracy of Training data is: {accuracy_train:.2f}")

print(confusion_matrix(Y_Train,Y_pred_Train))

# Generate a classification report
report = classification_report(Y_Train, Y_pred_Train)
print(f"Classification Report:\n{report}")

Y_pred_Test=MNB.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Training data is: 0.71
[[9542   36]
 [3963  459]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83      9578
           1       0.93      0.10      0.19      4422

    accuracy                           0.71     14000
   macro avg       0.82      0.55      0.51     14000
weighted avg       0.78      0.71      0.62     14000

Accuracy of Test data is: 0.71
[[4082   23]
 [1745  150]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.99      0.82      4105
           1       0.87      0.08      0.15      1895

    accuracy                           0.71      6000
   macro avg       0.78      0.54      0.48      6000
weighted avg       0.75      0.71      0.61      6000



In [19]:
modelgbc = GradientBoostingClassifier(random_state=45,n_estimators=100)
modelgbc.fit(X_Train, Y_Train)

Y_pred_Train=modelgbc.predict(X_Train)
accuracy_train = accuracy_score(Y_Train, Y_pred_Train)
print(f"Accuracy of Training data is: {accuracy_train:.2f}")

print(confusion_matrix(Y_Train,Y_pred_Train))

# Generate a classification report
report = classification_report(Y_Train, Y_pred_Train)
print(f"Classification Report:\n{report}")

Y_pred_Test=modelgbc.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Training data is: 0.78
[[9205  373]
 [2684 1738]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.96      0.86      9578
           1       0.82      0.39      0.53      4422

    accuracy                           0.78     14000
   macro avg       0.80      0.68      0.69     14000
weighted avg       0.79      0.78      0.75     14000

Accuracy of Test data is: 0.76
[[3896  209]
 [1209  686]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85      4105
           1       0.77      0.36      0.49      1895

    accuracy                           0.76      6000
   macro avg       0.76      0.66      0.67      6000
weighted avg       0.76      0.76      0.73      6000



In [20]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
skf = StratifiedKFold(n_splits=5)  # n_splits defines the number of folds


In [21]:
model = RandomForestClassifier(random_state=45,n_estimators=1000)

In [22]:
scores = cross_val_score(model,dense,df_y_enc,cv=skf)
print(f'Cross-validation scores: {scores}')



C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-

Cross-validation scores: [0.8115  0.781   0.787   0.737   0.76825]


In [23]:
print(f'Average Cross Validation score is: {np.mean(scores)}')

Average Cross Validation score is: 0.77695
